# Processing the Corpus with Gensim

2019-02-13

In [159]:
#!pip3 install --user snowballstemmer

In [269]:
import gzip
import itertools as it
import logging
import glob
import os
import re

from concurrent import futures
from gensim import corpora, parsing

In [176]:
logging.basicConfig(level=logging.DEBUG)

In [265]:
class ArxivCorpus(corpora.TextCorpus):
    """
    Corpus from arxiv.org source files.
    
    By subclassing corpora.TextCorpus we get much of the wiring for free:
    dictionary support, bag-of-words translation, etc. We can also save to
    other corpus formats.
    
    """
    length = None
    parallel_get_texts = False
    
    def __init__(self, parallel=False, *args, **kwargs):
        """
        Bit of a hack to set up parallel=True/False, useful for running
        get_texts() always in parallel.
        
        """
        if parallel:
            self.parallel_get_texts = True
        super().__init__(*args, **kwargs)
    
    def __len__(self):
        """Use self.getstream() to count documents"""
        if self.length is not None:
            return self.length
        return sum(1 for _ in self.get_file_list())
    
    def get_file_list(self):
        """Walk through self.input and yield document paths"""
        RE_TXTGZ = re.compile(r'\.txt\.gz$', re.I)
        
        if self.input is None:
            raise ValueError('self.input must be specified')
        if isinstance(self.input, str):
            self.input = [self.input]
        if any(not os.path.isdir(s) for s in self.input):
            raise OSError('all self.input values should be directories')
        
        num_files = 0
        for indir in self.input:
            for pwd, dirs, files in os.walk(indir):
                text_files = [f for f in files if RE_TXTGZ.search(f) is not None]
                for file in text_files:
                    yield os.path.abspath(os.path.join(pwd, file))
                    num_files += 1
        
        if self.length is None:
            logging.info('Caching the number of documents')
            self.length = num_files
    
    def getstream(self):
        """Walk through self.get_file_list() and yield text"""
        for file in self.get_file_list():
            with gzip.GzipFile(file) as infil:
                yield infil.read().decode()
    
    def get_texts(self, parallel=False):
        """Walk through self.getstream() and yield documents"""
        docs = self.getstream()
        if self.metadata:
            raise NotImplementedError
        else:
            if parallel or self.parallel_get_texts:
                logging.info('Using experimental parallel get_stream()')
                jobs = []
                with futures.ProcessPoolExecutor() as ppe:
                    for doc in docs:
                        jobs += [ppe.submit(self.preprocess_text, doc)]
                    for doc in futures.as_completed(jobs):
                        yield doc.result()
            else:
                for doc in docs:
                    yield self.preprocess_text(doc)
    
    @staticmethod
    def preprocess_text(text):
        """
        Full preprocessing pipeline.
        
          1. Tokenize
          2. Normalize
          3. Remove stop words
          4. Stem the tokens
        
        """
        stemmer = parsing.PorterStemmer()
        
        RE_SPLIT_FIRST_PASS = re.compile(r'[\s,]+')
        RE_SPLIT_SECOND_PASS = re.compile(r'[-/\\(){}–—,\[\]]+')
        
        RE_HYPHEN_COLLAPSE = [
            {'search': re.compile(r'x[-—]ray'),          'replace': 'xray'},
            {'search': re.compile(r'e[-—]mail'),         'replace': 'email'},
            {'search': re.compile(r'non[-—]([a-zA-Z])'), 'replace': 'non\\1'},
            {'search': re.compile(r'space[-—]time'),     'replace': 'spacetime'},    
        ]
        
        RE_BRACKET_DETRITUS = re.compile(r'[^\[\]]+\[[^\[\]]+')
        RE_LATEX_SEC = re.compile(r'(fig|eqn?)[:a-zA-Z0-9]')
        RE_EDGE_PUNCT = re.compile(r'(?<=^)[^a-zA-Z0-9]+|[^a-zA-Z0-9]+(?=$)')
        RE_ALPHANUM = re.compile(r'[0-9a-zA-Z]')
        RE_POSSESSION = re.compile(r'[\'‘’`][sS]$')
        RE_NUM = re.compile(r'[0-9]')
        
        # split on whitespace and remove whitespace-only tokens
        # normalize to ascii and convert to lowercase
        stream = [
            corpora.textcorpus.deaccent(w).lower()
            for w in RE_SPLIT_FIRST_PASS.split(text)
        ]
        
        # collapse commonly-hyphenated words
        for regex in RE_HYPHEN_COLLAPSE:
            stream = [regex['search'].sub(regex['replace'], s) for s in stream]
            
        # Remove bracket detritus like blah[blah]
        stream = [s for s in stream if RE_BRACKET_DETRITUS.search(s) is None]

        # Secondary split: hyphens, slashes, parens, braces, etc.
        # https://stackoverflow.com/a/952952/656740
        stream = list(filter(None, it.chain.from_iterable(
            RE_SPLIT_SECOND_PASS.split(s)
            for s in stream
        )))
        
        # remove common LaTeX labels like fig:blah or eq:blah1
        stream = [s for s in stream if RE_LATEX_SEC.search(s) is None]

        # strip edge punctuation
        stream = [RE_EDGE_PUNCT.sub('', s) for s in stream]

        # do not include nonword tokens
        stream = [s for s in stream if RE_ALPHANUM.search(s) is not None]
        
        # remove email addresses
        stream = [s for s in stream if s.find('@') == -1]

        # strip possession
        stream = [RE_POSSESSION.sub('', s) for s in stream]
        
        # remove leftover colon-y things like 'sec:my_great_section'
        stream = [s for s in stream if s.find(':') == -1]
        
        # remove numbers
        stream = [s for s in stream if RE_NUM.search(s) is None]
        
        # remove obvious stopwords
        stream = [s for s in stream if s not in corpora.textcorpus.STOPWORDS]
        
        # stem
        stream = [stemmer.stem(s) for s in stream]
        
        return stream
    
    def init_dictionary(self, dictionary):
        """
        This is a copy-and-paste job from gensim.corpora.textcorpus.TextCorpus.
        I needed this to set up creating the dictionary in parallel.

        """
        self.dictionary = dictionary if dictionary is not None else corpora.Dictionary()
        if self.input is not None:
            if dictionary is None:
                logging.info('Initializing dictionary')
                metadata_setting = self.metadata
                self.metadata = False
                self.dictionary.add_documents(self.get_texts(parallel=True))
                self.metadata = metadata_setting
            else:
                logging.info('Input stream provided but dictionary already initialized')
        else:
            logging.warning(
                'No input document stream provided;',
                'assuming dictionary will be initialized some other way.'
            )

In [274]:
corpus = ArxivCorpus(input=glob.glob('../../data/arxiv/text/9*'))

INFO:root:Initializing dictionary
INFO:root:Using experimental parallel get_stream()
INFO:root:Caching the number of documents
INFO:gensim.corpora.dictionary:adding document #0 to Dictionary(0 unique tokens: [])
INFO:gensim.corpora.dictionary:adding document #10000 to Dictionary(115416 unique tokens: ['a.d', 'a.i', 'a.r', 'a.w', 'abil']...)
INFO:gensim.corpora.dictionary:adding document #20000 to Dictionary(169372 unique tokens: ['a.d', 'a.i', 'a.r', 'a.w', 'abil']...)
INFO:gensim.corpora.dictionary:adding document #30000 to Dictionary(214007 unique tokens: ['a.d', 'a.i', 'a.r', 'a.w', 'abil']...)
INFO:gensim.corpora.dictionary:adding document #40000 to Dictionary(255322 unique tokens: ['a.d', 'a.i', 'a.r', 'a.w', 'abil']...)
INFO:gensim.corpora.dictionary:adding document #50000 to Dictionary(293871 unique tokens: ['a.d', 'a.i', 'a.r', 'a.w', 'abil']...)
INFO:gensim.corpora.dictionary:adding document #60000 to Dictionary(331385 unique tokens: ['a.d', 'a.i', 'a.r', 'a.w', 'abil']...)
IN